In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import warnings
warnings.filterwarnings(action='ignore')

import folium
KC_coord = (47.4081,-121.9949)
Seattle_coord = (47.6062,-122.3321)
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-notebook')
#from yellowbrick.regressor import ResidualsPlot

import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OrdinalEncoder, OneHotEncoder

In [ ]:
df = pd.read_csv('./data/kc_house_data.csv', index_col=0, parse_dates=['date']) 
# specify index_col=0 to avoid creating an "Unnamed: 0" column.
# specify parse date to avoid object dtype for dates
df

In [ ]:
len(df)

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr());

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
waterfront_count = df['waterfront'].value_counts()
waterfront_count

In [ ]:
waterfront_proportions = df['waterfront'].value_counts()[1] / df['waterfront'].value_counts()[0]
waterfront_proportions

In [ ]:
watefront_null_proportion = df['waterfront'].isna().sum() * waterfront_proportions
watefront_null_proportion

In [ ]:
waterfront_yes = df.loc[ df['waterfront'] == 'YES' ]

waterfront_avg_yes_price = np.mean( waterfront_yes['price'] )
waterfront_med_yes_price = np.median( waterfront_yes['price'] )

waterfront_null = df[df['waterfront'].isna()]
waterfront_null.loc[~waterfront_null.index.duplicated(), :]

In [ ]:
waterfront_null_avg_filter = waterfront_null.loc[ waterfront_null['price'] >= waterfront_avg_yes_price ]
waterfront_null_med_filter = waterfront_null.loc[ waterfront_null['price'] >= waterfront_med_yes_price ]
waterfront_med = np.median(waterfront_null_med_filter['price'])


waterfront_null_avg_count = len(waterfront_null_avg_filter)  
waterfront_null_med_count = len(waterfront_null_med_filter) 

waterfront_convert = waterfront_null_med_count * waterfront_proportions

print(f"""mean: {np.round(waterfront_avg_yes_price, 2)}, count: {waterfront_null_avg_count} 

median: {waterfront_med_yes_price}, count: {waterfront_null_med_count}

waterfront_convert: {waterfront_convert}

median waterfront homes >= waterfront_med_yes_price: {waterfront_med}

{waterfront_null_med_filter}""")

In [ ]:
view_count = df['view'].value_counts()
view_count

In [ ]:
condition_count = df['condition'].value_counts()
condition_count

In [ ]:
bedroom_count = df['bedrooms'].value_counts()
bedroom_count

# Data Preparation

In [ ]:
#drop the 33 bedroom outlier 
df = df[df['bedrooms'] != 33]

In [ ]:
df['bedrooms_bin']=pd.cut(df['bedrooms'], bins = [0,1,2,3,4,5,6,99], labels=['1 bed','2 bed','3 bed','4 bed','5 bed','6 bed','7+ bed'])
df.head()

In [ ]:
#drop the 33 bedroom outlier 
df.drop(['date', 'yr_renovated', 'sqft_basement'], axis=1, inplace=True)
df.head()


# Taking Out Outliers

### Price Outliers

In [ ]:
# calculate 25, 75, iqr, whiskers for 'price'
price_q25, price_q75 = np.percentile(df['price'], [25, 75])
price_iqr = price_q75 - price_q25
price_min = price_q25 - (1.5 * price_iqr)
price_max = price_q75 + (1.5 * price_iqr)

print(f"""price_min: {price_min} 
price_max: {price_max}""")

In [ ]:
# set df to include everything within the whiskers except the outliers
df = df[df['price'] >= price_min]
df = df[df['price'] <= price_max]

### Sqft Lot Outliers

In [ ]:
# calculate 25, 75, iqr, whiskers for 'sqft_lot'
sqft_lot_q25, sqft_lot_q75 = np.percentile(df['sqft_lot'], [25, 75])
sqft_lot_iqr = sqft_lot_q75 - sqft_lot_q25
sqft_lot_min = sqft_lot_q25 - (1.5 * sqft_lot_iqr)
sqft_lot_max = sqft_lot_q75 + (1.5 * sqft_lot_iqr)


print(f"""sqft_lot_min: {sqft_lot_min} 
sqft_lot_max: {sqft_lot_max}""")

In [ ]:
# set df to include everything within the whiskers except the outliers
df = df[df['sqft_lot'] >= sqft_lot_min]
df = df[df['sqft_lot'] <= sqft_lot_max]

### Grade Outliers

In [ ]:
# check value counts of grades
grade_count = print(df['grade'].value_counts())

In [ ]:
df = df[df['grade'] != '3 Poor']
df = df[df['grade'] != '12 Luxury']

ohe executes 2 transformations, test and train. data for any given input variable: First, we use OneHotEncoder to split up our categorical variables. Second, we run a linear regression on encoded values using .fit(). Then, it takes in a training df ('df'), a test df ('df2') and a column name. Next, it returns the training df and the test df concatenated with newly encoded columns. Lastly, we return a list of these newly encoded column names. 

In [ ]:
def ohe(df, df2, column):
    for col in column:
        train = df[[col]]
        ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown="error")
        ohe.fit(train)
        encoded_train = ohe.transform(train)
        col_labels = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_train = pd.DataFrame(encoded_train, columns=col_labels, index=df.index)
        df = pd.concat([df, encoded_train], axis=1)
        
        test = df2[[col]]
        encoded_test = ohe.transform(test)
        col_labels = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_test = pd.DataFrame(encoded_test, columns=col_labels, index=df2.index)
        df2 = pd.concat([df2, encoded_test], axis=1)
        
    return df, df2, encoded_train.columns.tolist()

We run a Standard Scaler to scale all the values in the test data and training data so that our models results are in comparable units. Then, we score the scaled data to get the R2 to assess the strength of our model.

In [ ]:
def scale_score(x, y, x2, y2):
    scaler = StandardScaler()
    scaler.fit(x)
    X_train_scaled = scaler.transform(x)
    X_test_scaled = scaler.transform(x2)
    
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)
    
    train_pred = model.predict(X_train_scaled)
    test_pred = model.predict(X_test_scaled)
    
    print(f"""training score: {r2_score(y_train, train_pred)}
    test score: {r2_score(y_test, test_pred)}""")
    
    return X_train_scaled, X_test_scaled, train_pred, test_pred

Uses price and predictions to calculate Mean Absolute Error (MAE), Mean Squared Error (MSE) and Root Mean Squared Error (RSME) for the training and testing data sets. We will compare these values to evaluate model performance.

In [ ]:
def get_train_test_metrics(y_train,train_preds,
               y_test,test_preds
              ):
    """
    Prints different training and testing metrics, namely R2, MAE, MSE, RMSE
    """
    print("\nTraining Metrics:")
    print(f"R2: {r2_score(y_train, train_preds):.3f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_train, train_preds):.3f}")
    print(f"Mean Squared Error: {mean_squared_error(y_train, train_preds):.3f}")
    print(f"Root Mean Squared Error: {mean_squared_error(y_train, train_preds, squared=False):.3f}")
    
    print("\nTesting Metrics:")
    print(f"R2: {r2_score(y_test, test_preds):.3f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_test, test_preds):.3f}")
    print(f"Mean Squared Error: {mean_squared_error(y_test, test_preds):.3f}")
    print(f"Root Mean Squared Error: {mean_squared_error(y_test, test_preds, squared=False):.3f}")

In [ ]:
y = df['price']
X = df.drop('price', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.25)

### Condition

In [ ]:
X_train, X_test, encoded_condition = ohe(X_train, X_test, ['condition'])

In [ ]:
X_train['condition'] = X_train['condition'].map({'Very Good': 5, 'Good': 4, 'Average': 3, 'Fair': 2, 'Poor': 1})

In [ ]:
X_test['condition'] = X_test['condition'].map({'Very Good': 5, 'Good': 4, 'Average': 3, 'Fair': 2, 'Poor': 1})

### View

In [ ]:
X_train['view'].isna().sum()

In [ ]:
X_train['view'].fillna('NONE', inplace=True)
X_test['view'].fillna('NONE', inplace=True)

In [ ]:


X_train, X_test, encoded_view = ohe(X_train, X_test, ['view'])

In [ ]:
X_train['view'] = X_train['view'].map({'EXCELLENT': 5, 'GOOD': 4, 'AVERAGE': 3, 'FAIR': 2, 'NONE': 1})

In [ ]:
X_test['view'] = X_test['view'].map({'EXCELLENT': 5, 'GOOD': 4, 'AVERAGE': 3, 'FAIR': 2, 'NONE': 1})

### Waterfront

In [ ]:
X_train['waterfront'].fillna('NO', inplace=True)
X_test['waterfront'].fillna('NO', inplace=True)

In [ ]:
waterfront_train = X_train[['waterfront']]

encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_train)
encoder_waterfront.categories_[0]
waterfront_encoded_train = encoder_waterfront.transform(waterfront_train)
waterfront_encoded_train = waterfront_encoded_train.flatten()

X_train['waterfront'] = waterfront_encoded_train

In [ ]:
waterfront_test = X_test[['waterfront']]

encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_test)
encoder_waterfront.categories_[0]
waterfront_encoded_test = encoder_waterfront.transform(waterfront_test)
waterfront_encoded_test = waterfront_encoded_train.flatten()

X_train['waterfront'] = waterfront_encoded_test

### Bedroom Bins

In [ ]:
X_train, X_test, encoded_bedrooms_bins = ohe(X_train, X_test, ['bedrooms_bin'])

### Grade

In [ ]:
X_train, X_test, encoded_grade = ohe(X_train, X_test, ['grade'])

### Zipcode

In [ ]:
X_train, X_test, encoded_zipcode = ohe(X_train, X_test, ['zipcode'])

In [ ]:
dummy = DummyRegressor()
dummy.fit(X_train, y_train)
dummy.score(X_train, y_train)

In [ ]:
cols = ['sqft_living']
y = y_train
x = X_train.filter(cols, axis=1)
y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
y_sqft_liv = df['price']
X_sqft_liv = df['sqft_living']

model = sm.OLS(y_sqft_liv, sm.add_constant(X_sqft_liv)).fit()
model_summary = model.summary()
model_summary

In [ ]:
type(X)

In [ ]:
df.corr()['price'].abs().sort_values(ascending=False)

In [ ]:
df['price']

# start

In [ ]:
df = pd.read_csv('./data/kc_house_data.csv', index_col=0, parse_dates=['date'])

In [ ]:
#Creating a heatmap to see correlations
plt.figure(figsize=(12, 6))
mask = np.triu(np.ones_like(df.corr(), dtype=bool))
heatmap = sns.heatmap(df.corr(), mask=mask, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

In [ ]:
highcor = ['bathrooms', 'sqft_above','sqft_living15','sqft_lot15','grade']
relcols = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'floors', 'sqft_above', 'sqft_living15']
sns.pairplot(df, corner=True);

In [ ]:
#Created a functions to get models, summaries, and encode data
def getmodel(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    mod = LinearRegression()
    mod.fit(X_train, y_train)
    pred = mod.predict(X_test)
 
    print(f'y intercept: {mod.intercept_}')
    print(f'slope: {mod.coef_}')
    print(f'R2: %.3f' %r2_score(y_test, pred))
    print(f'mean2 error: %.3f' %mean_squared_error(y_test, pred))
    print(f'predicted path: {pred}')
    
    Xrline = mod.coef_*X_test+mod.intercept_
    plt.scatter(y_test, pred, alpha=0.25);
    plt.plot(pred, Xrline);
    
    
    
    
def getols(x,y):
    
    x = sm.add_constant(x)
    result = sm.OLS(y, x).fit()
    print(result.summary())
    
    

    
def odinalencode(x):
    encode_x = OrdinalEncoder()

    encode_x.fit(x)

    x_encoded = encode_x.transform(x)
    x_encoded = x_encoded.flatten()
    return x_encoded

In [ ]:
df['waterfront'] = df['waterfront'].replace(np.nan, 'NO')
df['waterfront'] = odinalencode(df[['waterfront']])

viewdict = {'NONE':1,'AVERAGE':2,'GOOD':3,'FAIR':4,'EXCELLENT':5}
df['view'] = df['view'].map(viewdict)

condict = {'Poor':1,'Fair':2,'Fair':3,'Average':4,'Good':5,'Very Good':6}
df['condition'] = df['condition'].map(condict)

gradedict = {'3 Poor':1,'4 Low':2,'5 Fair':3,'6 Low Average':4,'7 Average':7,'8 Good':8,'9 Better':9,'10 Very Good':10,'11 Excellent':11,'12 Luxury':12,'13 Mansion':13}
df['grade'] = df['grade'].map(gradedict)

df['sqft_basement'] = df['sqft_basement'].map({'?':0})
df['sqft_basement'] = df['sqft_basement'].replace(np.nan, 0)

df['yr_renovated'] = df['yr_renovated'].replace(np.nan, 0)

df.drop('view',axis=1,inplace=True)
df.drop('date',axis=1,inplace=True)

In [ ]:
bathModel = getmodel(df[['price']], df['bathrooms'])
bathols = getols(df[['price']], df['bathrooms'])

In [ ]:
sqft_liv = getmodel(df[['price']], df['sqft_living'])        
sqft_livols = getols(df[['price']], df['sqft_living'])

In [ ]:

sqft15 = getmodel(df[['price']], df['sqft_living15'])
sqftols = getols(df[['price']], df['sqft_living15'])

In [ ]:
X = df.drop(['price'],axis=1).values
y = df['price'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)
reg = LinearRegression()
reg.fit(X_train,y_train)

pred = reg.predict(X_test)

print(f'r2 score: {r2_score(y_test, pred)}')

plt.scatter(y_test,pred,alpha=0.25);
plt.xlabel('Actual')
plt.ylabel('Predicted');

In [ ]:
newdf = df.copy()
#Dropping columns for colinearity
newdf.drop(highcor, axis=1, inplace=True)
newdf.head()

In [ ]:
X = newdf.drop(['price'],axis=1).values
y = newdf['price'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

reg.fit(X_train,y_train)

pred = reg.predict(X_test)

print(f'r2 score: {r2_score(y_test, pred)}')
#getols(y_test,pred)
plt.scatter(y_test,pred,alpha=0.25);
plt.xlabel('Actual')
plt.ylabel('Predicted');

In [ ]:
# Creating a fuction to find mean price per zipcode
def zipmean(x):
    y = df[df['zipcode'] == x]
    m = int(y["price"].mean())
    return m

In [ ]:
df['zipcode'].value_counts()

In [ ]:
zipmean(98103)

In [ ]:
#Creating a list for unique zipcodes and for their average home value
ziplist = []
zipprice = []
for i in df['zipcode']:
    b = zipmean(i)
    if b in zipprice:
        continue
    else:
        zipprice.append(b)
    if i in ziplist:
        continue
    else:
        ziplist.append(i)

In [ ]:
#Testing results to make sure they equal in length
print(len(ziplist))
print(len(zipprice))

In [ ]:
#Creating list of zipcode price mean values from original dataframe
avg_zip_price = []
for i in df['zipcode']:
    m = zipmean(i)
    avg_zip_price.append(m)

In [ ]:
#Creating column for average home price per zipcode
df['avg_zip_price'] = avg_zip_price

In [ ]:
#Ranking zipcodes based on their average home prices
zip_rank = []
for i in df['avg_zip_price']:
    if i >= 1600000:
        zip_rank.append(1)
    elif i >= 1200000:
        zip_rank.append(2)
    elif i >= 800000:
        zip_rank.append(3)  
    elif i >= 400000:
        zip_rank.append(4) 
    else:
        zip_rank.append(5)

In [ ]:
#Creating Column for zipcode ranks
df['zip_rank'] = zip_rank

In [ ]:
#Checking column for correct values
df['zip_rank'].value_counts()

In [ ]:
X = newdf.drop(['price'],axis=1).values
y = newdf['price'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

reg.fit(X_train,y_train)

pred = reg.predict(X_test)

print(f'r2 score: {r2_score(y_test, pred)}')
#getols(y_test,pred)
plt.scatter(y_test,pred,alpha=0.25);
plt.xlabel('Actual')
plt.ylabel('Predicted');



# Visualization

In [ ]:
#Creating heatmap style seaborn scatterplot based on average price per location
x = df['long']
y = df['lat']
h = df['avg_zip_price']
sns.scatterplot(x,y,hue=h).set(title='Zipcodes by home value');

In [ ]:
#creating seaborn scatterplot for different zipcodes
sns.scatterplot(x=df['long'],y=df['lat'],hue=df['zipcode']).set(title='Map by zipcodes');

In [ ]:
#Creating heatmap style MPL scatterplot based on average price per location
colors = df['zip_rank']

lon = df['long']
lat = df['lat']
plt.scatter(lon,lat,c=colors,alpha=0.5);
plt.xlabel('Longitude');
plt.ylabel('Latitude');
plt.title('MPL plot by zip_avg_price');

In [ ]:
#creating MPL scatterplot for different zipcodes
colors = df['zipcode']

lon = df['long']
lat = df['lat']
plt.scatter(lon,lat,c=colors,alpha=0.5);
plt.title('Map of houses by zipcode')
plt.xlabel('Longitude');
plt.ylabel('Latitude');

In [ ]:
x = df['long']
y = df['lat']
h = df['sqft_living']
sns.scatterplot(x,y,hue=h).set(title='Zipcodes by home sqft_living');

In [ ]:
x = df['long']
y = df['lat']
h = df['grade']
sns.scatterplot(x,y,hue=h).set(title='Zipcodes by home grade');

In [ ]:
x = df['long']
y = df['lat']
h = df['price']
sns.scatterplot(x,y,hue=h).set(title='Zipcodes by individual home value');